In [1]:
import random
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Let's read the mnist dataset
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In this exercise your task is to fill in the gaps in this code by implementing the backpropagation algorithm
Once this is done, you can run the network on the MNIST example and see how it performs. Feel free to play with the parameters.

If you found this task too easy, try to implement a "fully vectorized" version, i.e. one using matrix operations instead of going over examples one by one.

In [27]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a single case
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    def update_mini_batch(self, mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch which is as in tensorflow API.
        # eta is the learning rate
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in zip(mini_batch[0],mini_batch[1]):
            delta_nabla_b, delta_nabla_w = self.backprop(x.reshape(784,1), y.reshape(10,1))
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch[0]))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch[0]))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
    def backprop(self, x, y):
        # For a single input (x,y) return a pair of lists.
        # First contains gradients over biases, second over weights.

        # First initialize the list of gradient arrays
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Then go forward remembering all values before and after activations
        # in two other array lists
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # Now go backward from the final cost applying backpropagation
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(z[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) 
        
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        test_results = [(np.argmax(self.feedforward(test_data[0][i].reshape(784,1))), np.argmax(test_data[1][i]))
                        for i in range(len(test_data[0]))]
        #print test_results
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        for j in xrange(epochs):
            self.update_mini_batch(training_data.next_batch(mini_batch_size), eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data.next_batch(mini_batch_size)), mini_batch_size)
            else:
                print "Epoch {0} complete".format(j)


network = Network([784,30,10])
network.SGD(mnist.train,epochs=10000,mini_batch_size=100,eta=3.0,test_data=mnist.test)



Epoch 0: 9 / 100
Epoch 1: 4 / 100
Epoch 2: 11 / 100
Epoch 3: 10 / 100
Epoch 4: 12 / 100
Epoch 5: 12 / 100
Epoch 6: 12 / 100
Epoch 7: 16 / 100
Epoch 8: 7 / 100
Epoch 9: 10 / 100
Epoch 10: 8 / 100
Epoch 11: 21 / 100
Epoch 12: 14 / 100
Epoch 13: 14 / 100
Epoch 14: 15 / 100
Epoch 15: 15 / 100
Epoch 16: 19 / 100
Epoch 17: 16 / 100
Epoch 18: 14 / 100
Epoch 19: 14 / 100
Epoch 20: 23 / 100
Epoch 21: 10 / 100
Epoch 22: 13 / 100
Epoch 23: 14 / 100
Epoch 24: 18 / 100
Epoch 25: 8 / 100
Epoch 26: 17 / 100
Epoch 27: 15 / 100
Epoch 28: 8 / 100
Epoch 29: 17 / 100
Epoch 30: 9 / 100
Epoch 31: 21 / 100
Epoch 32: 14 / 100
Epoch 33: 15 / 100
Epoch 34: 12 / 100
Epoch 35: 14 / 100
Epoch 36: 24 / 100
Epoch 37: 14 / 100
Epoch 38: 22 / 100
Epoch 39: 23 / 100
Epoch 40: 24 / 100
Epoch 41: 24 / 100
Epoch 42: 6 / 100
Epoch 43: 27 / 100
Epoch 44: 22 / 100
Epoch 45: 26 / 100
Epoch 46: 24 / 100
Epoch 47: 16 / 100
Epoch 48: 16 / 100
Epoch 49: 20 / 100
Epoch 50: 21 / 100
Epoch 51: 15 / 100
Epoch 52: 25 / 100
Epoch 53: 2

Epoch 419: 66 / 100
Epoch 420: 62 / 100
Epoch 421: 69 / 100
Epoch 422: 59 / 100
Epoch 423: 54 / 100
Epoch 424: 64 / 100
Epoch 425: 64 / 100
Epoch 426: 64 / 100
Epoch 427: 62 / 100
Epoch 428: 58 / 100
Epoch 429: 70 / 100
Epoch 430: 64 / 100
Epoch 431: 64 / 100
Epoch 432: 70 / 100
Epoch 433: 64 / 100
Epoch 434: 61 / 100
Epoch 435: 69 / 100
Epoch 436: 53 / 100
Epoch 437: 62 / 100
Epoch 438: 61 / 100
Epoch 439: 70 / 100
Epoch 440: 68 / 100
Epoch 441: 60 / 100
Epoch 442: 63 / 100
Epoch 443: 56 / 100
Epoch 444: 52 / 100
Epoch 445: 58 / 100
Epoch 446: 67 / 100
Epoch 447: 61 / 100
Epoch 448: 59 / 100
Epoch 449: 70 / 100
Epoch 450: 63 / 100
Epoch 451: 62 / 100
Epoch 452: 60 / 100
Epoch 453: 62 / 100
Epoch 454: 62 / 100
Epoch 455: 60 / 100
Epoch 456: 66 / 100
Epoch 457: 62 / 100
Epoch 458: 64 / 100
Epoch 459: 59 / 100
Epoch 460: 61 / 100
Epoch 461: 65 / 100
Epoch 462: 73 / 100
Epoch 463: 66 / 100
Epoch 464: 71 / 100
Epoch 465: 61 / 100
Epoch 466: 61 / 100
Epoch 467: 55 / 100
Epoch 468: 69 / 100


Epoch 842: 70 / 100
Epoch 843: 68 / 100
Epoch 844: 69 / 100
Epoch 845: 67 / 100
Epoch 846: 69 / 100
Epoch 847: 75 / 100
Epoch 848: 67 / 100
Epoch 849: 71 / 100
Epoch 850: 74 / 100
Epoch 851: 65 / 100
Epoch 852: 76 / 100
Epoch 853: 65 / 100
Epoch 854: 73 / 100
Epoch 855: 75 / 100
Epoch 856: 68 / 100
Epoch 857: 68 / 100
Epoch 858: 67 / 100
Epoch 859: 73 / 100
Epoch 860: 71 / 100
Epoch 861: 66 / 100
Epoch 862: 62 / 100
Epoch 863: 64 / 100
Epoch 864: 78 / 100
Epoch 865: 63 / 100
Epoch 866: 76 / 100
Epoch 867: 75 / 100
Epoch 868: 65 / 100
Epoch 869: 73 / 100
Epoch 870: 69 / 100
Epoch 871: 78 / 100
Epoch 872: 71 / 100
Epoch 873: 70 / 100
Epoch 874: 70 / 100
Epoch 875: 70 / 100
Epoch 876: 72 / 100
Epoch 877: 73 / 100
Epoch 878: 70 / 100
Epoch 879: 74 / 100
Epoch 880: 78 / 100
Epoch 881: 71 / 100
Epoch 882: 74 / 100
Epoch 883: 65 / 100
Epoch 884: 67 / 100
Epoch 885: 72 / 100
Epoch 886: 69 / 100
Epoch 887: 69 / 100
Epoch 888: 64 / 100
Epoch 889: 73 / 100
Epoch 890: 63 / 100
Epoch 891: 73 / 100


Epoch 1244: 71 / 100
Epoch 1245: 71 / 100
Epoch 1246: 73 / 100
Epoch 1247: 71 / 100
Epoch 1248: 77 / 100
Epoch 1249: 71 / 100
Epoch 1250: 71 / 100
Epoch 1251: 71 / 100
Epoch 1252: 69 / 100
Epoch 1253: 68 / 100
Epoch 1254: 77 / 100
Epoch 1255: 74 / 100
Epoch 1256: 72 / 100
Epoch 1257: 69 / 100
Epoch 1258: 79 / 100
Epoch 1259: 72 / 100
Epoch 1260: 76 / 100
Epoch 1261: 75 / 100
Epoch 1262: 71 / 100
Epoch 1263: 73 / 100
Epoch 1264: 58 / 100
Epoch 1265: 75 / 100
Epoch 1266: 75 / 100
Epoch 1267: 71 / 100
Epoch 1268: 70 / 100
Epoch 1269: 57 / 100
Epoch 1270: 73 / 100
Epoch 1271: 72 / 100
Epoch 1272: 77 / 100
Epoch 1273: 72 / 100
Epoch 1274: 75 / 100
Epoch 1275: 75 / 100
Epoch 1276: 78 / 100
Epoch 1277: 72 / 100
Epoch 1278: 64 / 100
Epoch 1279: 80 / 100
Epoch 1280: 63 / 100
Epoch 1281: 72 / 100
Epoch 1282: 73 / 100
Epoch 1283: 78 / 100
Epoch 1284: 72 / 100
Epoch 1285: 64 / 100
Epoch 1286: 69 / 100
Epoch 1287: 70 / 100
Epoch 1288: 83 / 100
Epoch 1289: 67 / 100
Epoch 1290: 68 / 100
Epoch 1291: 7

Epoch 1644: 73 / 100
Epoch 1645: 74 / 100
Epoch 1646: 80 / 100
Epoch 1647: 73 / 100
Epoch 1648: 73 / 100
Epoch 1649: 72 / 100
Epoch 1650: 71 / 100
Epoch 1651: 74 / 100
Epoch 1652: 69 / 100
Epoch 1653: 73 / 100
Epoch 1654: 73 / 100
Epoch 1655: 67 / 100
Epoch 1656: 76 / 100
Epoch 1657: 83 / 100
Epoch 1658: 66 / 100
Epoch 1659: 74 / 100
Epoch 1660: 74 / 100
Epoch 1661: 68 / 100
Epoch 1662: 79 / 100
Epoch 1663: 66 / 100
Epoch 1664: 79 / 100
Epoch 1665: 73 / 100
Epoch 1666: 80 / 100
Epoch 1667: 65 / 100
Epoch 1668: 72 / 100
Epoch 1669: 82 / 100
Epoch 1670: 79 / 100
Epoch 1671: 75 / 100
Epoch 1672: 70 / 100
Epoch 1673: 75 / 100
Epoch 1674: 74 / 100
Epoch 1675: 74 / 100
Epoch 1676: 68 / 100
Epoch 1677: 77 / 100
Epoch 1678: 81 / 100
Epoch 1679: 70 / 100
Epoch 1680: 64 / 100
Epoch 1681: 76 / 100
Epoch 1682: 74 / 100
Epoch 1683: 73 / 100
Epoch 1684: 79 / 100
Epoch 1685: 76 / 100
Epoch 1686: 71 / 100
Epoch 1687: 78 / 100
Epoch 1688: 71 / 100
Epoch 1689: 71 / 100
Epoch 1690: 75 / 100
Epoch 1691: 7

...

Epoch 8025: 76 / 100
Epoch 8026: 77 / 100
Epoch 8027: 81 / 100
Epoch 8028: 73 / 100
Epoch 8029: 78 / 100
Epoch 8030: 74 / 100
Epoch 8031: 77 / 100
Epoch 8032: 87 / 100
Epoch 8033: 79 / 100
Epoch 8034: 80 / 100
Epoch 8035: 82 / 100
Epoch 8036: 83 / 100
Epoch 8037: 78 / 100
Epoch 8038: 76 / 100
Epoch 8039: 79 / 100
Epoch 8040: 80 / 100
Epoch 8041: 81 / 100
Epoch 8042: 80 / 100
Epoch 8043: 85 / 100
Epoch 8044: 73 / 100
Epoch 8045: 66 / 100
Epoch 8046: 78 / 100
Epoch 8047: 84 / 100
Epoch 8048: 75 / 100
Epoch 8049: 76 / 100
Epoch 8050: 77 / 100
Epoch 8051: 77 / 100
Epoch 8052: 83 / 100
Epoch 8053: 74 / 100
Epoch 8054: 79 / 100
Epoch 8055: 74 / 100
Epoch 8056: 81 / 100
Epoch 8057: 79 / 100
Epoch 8058: 78 / 100
Epoch 8059: 85 / 100
Epoch 8060: 75 / 100
Epoch 8061: 74 / 100
Epoch 8062: 85 / 100
Epoch 8063: 80 / 100
Epoch 8064: 74 / 100
Epoch 8065: 86 / 100
Epoch 8066: 83 / 100
Epoch 8067: 88 / 100
Epoch 8068: 77 / 100
Epoch 8069: 72 / 100
Epoch 8070: 75 / 100
Epoch 8071: 80 / 100
Epoch 8072: 7

Epoch 8425: 83 / 100
Epoch 8426: 80 / 100
Epoch 8427: 77 / 100
Epoch 8428: 84 / 100
Epoch 8429: 79 / 100
Epoch 8430: 81 / 100
Epoch 8431: 84 / 100
Epoch 8432: 73 / 100
Epoch 8433: 70 / 100
Epoch 8434: 78 / 100
Epoch 8435: 75 / 100
Epoch 8436: 75 / 100
Epoch 8437: 75 / 100
Epoch 8438: 81 / 100
Epoch 8439: 74 / 100
Epoch 8440: 76 / 100
Epoch 8441: 77 / 100
Epoch 8442: 81 / 100
Epoch 8443: 82 / 100
Epoch 8444: 83 / 100
Epoch 8445: 80 / 100
Epoch 8446: 79 / 100
Epoch 8447: 85 / 100
Epoch 8448: 81 / 100
Epoch 8449: 81 / 100
Epoch 8450: 69 / 100
Epoch 8451: 74 / 100
Epoch 8452: 75 / 100
Epoch 8453: 77 / 100
Epoch 8454: 81 / 100
Epoch 8455: 78 / 100
Epoch 8456: 79 / 100
Epoch 8457: 78 / 100
Epoch 8458: 74 / 100
Epoch 8459: 75 / 100
Epoch 8460: 82 / 100
Epoch 8461: 80 / 100
Epoch 8462: 78 / 100
Epoch 8463: 76 / 100
Epoch 8464: 76 / 100
Epoch 8465: 83 / 100
Epoch 8466: 79 / 100
Epoch 8467: 80 / 100
Epoch 8468: 76 / 100
Epoch 8469: 83 / 100
Epoch 8470: 74 / 100
Epoch 8471: 82 / 100
Epoch 8472: 8

Epoch 8825: 81 / 100
Epoch 8826: 73 / 100
Epoch 8827: 88 / 100
Epoch 8828: 75 / 100
Epoch 8829: 77 / 100
Epoch 8830: 71 / 100
Epoch 8831: 77 / 100
Epoch 8832: 82 / 100
Epoch 8833: 81 / 100
Epoch 8834: 80 / 100
Epoch 8835: 81 / 100
Epoch 8836: 78 / 100
Epoch 8837: 82 / 100
Epoch 8838: 76 / 100
Epoch 8839: 77 / 100
Epoch 8840: 70 / 100
Epoch 8841: 75 / 100
Epoch 8842: 82 / 100
Epoch 8843: 73 / 100
Epoch 8844: 71 / 100
Epoch 8845: 86 / 100
Epoch 8846: 82 / 100
Epoch 8847: 82 / 100
Epoch 8848: 83 / 100
Epoch 8849: 89 / 100
Epoch 8850: 82 / 100
Epoch 8851: 77 / 100
Epoch 8852: 84 / 100
Epoch 8853: 77 / 100
Epoch 8854: 88 / 100
Epoch 8855: 76 / 100
Epoch 8856: 80 / 100
Epoch 8857: 77 / 100
Epoch 8858: 83 / 100
Epoch 8859: 80 / 100
Epoch 8860: 79 / 100
Epoch 8861: 80 / 100
Epoch 8862: 83 / 100
Epoch 8863: 83 / 100
Epoch 8864: 73 / 100
Epoch 8865: 81 / 100
Epoch 8866: 78 / 100
Epoch 8867: 70 / 100
Epoch 8868: 78 / 100
Epoch 8869: 73 / 100
Epoch 8870: 78 / 100
Epoch 8871: 82 / 100
Epoch 8872: 8

Epoch 9229: 79 / 100
Epoch 9230: 80 / 100
Epoch 9231: 71 / 100
Epoch 9232: 75 / 100
Epoch 9233: 86 / 100
Epoch 9234: 73 / 100
Epoch 9235: 75 / 100
Epoch 9236: 81 / 100
Epoch 9237: 80 / 100
Epoch 9238: 79 / 100
Epoch 9239: 81 / 100
Epoch 9240: 70 / 100
Epoch 9241: 81 / 100
Epoch 9242: 84 / 100
Epoch 9243: 85 / 100
Epoch 9244: 75 / 100
Epoch 9245: 84 / 100
Epoch 9246: 73 / 100
Epoch 9247: 73 / 100
Epoch 9248: 83 / 100
Epoch 9249: 84 / 100
Epoch 9250: 80 / 100
Epoch 9251: 74 / 100
Epoch 9252: 78 / 100
Epoch 9253: 86 / 100
Epoch 9254: 79 / 100
Epoch 9255: 78 / 100
Epoch 9256: 74 / 100
Epoch 9257: 78 / 100
Epoch 9258: 82 / 100
Epoch 9259: 79 / 100
Epoch 9260: 85 / 100
Epoch 9261: 78 / 100
Epoch 9262: 83 / 100
Epoch 9263: 78 / 100
Epoch 9264: 81 / 100
Epoch 9265: 71 / 100
Epoch 9266: 72 / 100
Epoch 9267: 84 / 100
Epoch 9268: 87 / 100
Epoch 9269: 82 / 100
Epoch 9270: 80 / 100
Epoch 9271: 85 / 100
Epoch 9272: 81 / 100
Epoch 9273: 79 / 100
Epoch 9274: 82 / 100
Epoch 9275: 80 / 100
Epoch 9276: 7

Epoch 9630: 82 / 100
Epoch 9631: 83 / 100
Epoch 9632: 71 / 100
Epoch 9633: 81 / 100
Epoch 9634: 78 / 100
Epoch 9635: 76 / 100
Epoch 9636: 80 / 100
Epoch 9637: 74 / 100
Epoch 9638: 74 / 100
Epoch 9639: 79 / 100
Epoch 9640: 84 / 100
Epoch 9641: 85 / 100
Epoch 9642: 79 / 100
Epoch 9643: 82 / 100
Epoch 9644: 81 / 100
Epoch 9645: 85 / 100
Epoch 9646: 77 / 100
Epoch 9647: 78 / 100
Epoch 9648: 80 / 100
Epoch 9649: 69 / 100
Epoch 9650: 82 / 100
Epoch 9651: 82 / 100
Epoch 9652: 80 / 100
Epoch 9653: 83 / 100
Epoch 9654: 82 / 100
Epoch 9655: 75 / 100
Epoch 9656: 83 / 100
Epoch 9657: 76 / 100
Epoch 9658: 81 / 100
Epoch 9659: 79 / 100
Epoch 9660: 76 / 100
Epoch 9661: 76 / 100
Epoch 9662: 81 / 100
Epoch 9663: 81 / 100
Epoch 9664: 78 / 100
Epoch 9665: 80 / 100
Epoch 9666: 78 / 100
Epoch 9667: 80 / 100
Epoch 9668: 82 / 100
Epoch 9669: 82 / 100
Epoch 9670: 81 / 100
Epoch 9671: 76 / 100
Epoch 9672: 83 / 100
Epoch 9673: 80 / 100
Epoch 9674: 72 / 100
Epoch 9675: 81 / 100
Epoch 9676: 77 / 100
Epoch 9677: 7

In [8]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fb235046e90>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fb234fef890>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fb200ffcad0>)

In [9]:
mnist.train

In [10]:
mnist.train.next_batch(10)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]))